In [259]:
import pandas as pd
import numpy as np
import math

from tqdm import tqdm
import time

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix

In [260]:
RANDOM_SEED = 13022022

In [261]:
new_df = pd.read_csv('df_first_draft.csv')

In [262]:
new_df.sample()

,Smiles,Active,train,len_smiles,C,O,c,1,2,[,...,7,8,e,A,K,M,g,i,L,9
6868,O=C1CCc2cc(OCCCCc3nnnn3C3CCCCC3)ccc2N1,NaN,0,38,13,2,7,2,2,0,...,0,0,0,0,0,0,0,0,0,0


In [263]:
train_new = new_df[new_df['train'] == 1]
test_new =new_df[new_df['train'] == 0]

In [264]:
X = train_new.drop(['Active', 'train', 'Smiles'], axis = 1)
y = train_new['Active']

In [265]:
X_test = test_new.drop(['Active', 'train', 'Smiles'], axis = 1)

In [266]:
X.sample()

,len_smiles,C,O,c,1,2,[,n,H,],...,7,8,e,A,K,M,g,i,L,9
4687,37,4,3,12,2,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
features = np.array(X)

In [268]:
test_features = np.array(X_test)

In [269]:
x_test = torch.tensor(test_features)
x_test = x_test.type(torch.FloatTensor)
x_test.size()

torch.Size([1614, 46])

In [270]:
x_train = torch.tensor(features)
x_train = x_train.type(torch.FloatTensor)
x_train.size()

torch.Size([5557, 46])

In [331]:
x_labels = torch.tensor(y)
x_labels = x_labels.type(torch.FloatTensor)
x_labels.size()

torch.Size([5557])

nX = 46
nH1 = 23
nH2 = 5
nY = 1
 
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH1, nH2, nY):        
        super(TwoLayersNet, self).__init__()     # конструктор предка с этим именем
         
        self.fc1 = nn.Linear(nX, nH1)             # создаём параметры модели
        #self.fc2 = nn.Linear(nH1, nH2)             # создаём параметры модели
        self.fc3 = nn.Linear(nH1, nY)             # в полносвязных слоях
          
    def forward(self, x):                        # задаётся прямой проход
        x = self.fc1(x)                          # выход первого слоя
                          # пропускаем через Sigmoid
        #x = self.fc2(x)                          # выход второго слоя
        x = nn.ReLU()(x)                   # пропускаем через сигмоид 
        x = self.fc3(x)                          # выход второго слоя
        x = nn.Sigmoid()(x)

        return x
          
model = TwoLayersNet(nX, nH1, nH2, nY)

In [344]:
import torch
import torch.nn as nn

from torch_geometric.nn import global_add_pool, radius
from torch_geometric.utils import remove_self_loops
from torch_sparse import SparseTensor
from torch_scatter import scatter

from layers import Global_MP, Local_MP
from utils import BesselBasisLayer, SphericalBasisLayer, MLP

class Config(object):
    def __init__(self, dim, n_layer, cutoff):
        self.dim = dim
        self.n_layer = n_layer
        self.cutoff = cutoff

class MXMNet(nn.Module):
    def __init__(self, config: Config, num_spherical=7, num_radial=6, envelope_exponent=5):
        super(MXMNet, self).__init__()

        self.dim = config.dim
        self.n_layer = config.n_layer
        self.cutoff = config.cutoff

        self.embeddings = nn.Parameter(torch.ones((5, self.dim)))

        self.rbf_l = BesselBasisLayer(16, 5, envelope_exponent)
        self.rbf_g = BesselBasisLayer(16, self.cutoff, envelope_exponent)
        self.sbf = SphericalBasisLayer(num_spherical, num_radial, 5, envelope_exponent)

        self.rbf_g_mlp = MLP([16, self.dim])
        self.rbf_l_mlp = MLP([16, self.dim])

        self.sbf_1_mlp = MLP([num_spherical * num_radial, self.dim])
        self.sbf_2_mlp = MLP([num_spherical * num_radial, self.dim])

        self.global_layers = torch.nn.ModuleList()
        for layer in range(config.n_layer):
            self.global_layers.append(Global_MP(config))

        self.local_layers = torch.nn.ModuleList()
        for layer in range(config.n_layer):
            self.local_layers.append(Local_MP(config))
        
        self.init()

    def init(self):
        stdv = math.sqrt(3)
        self.embeddings.data.uniform_(-stdv, stdv)

    def indices(self, edge_index, num_nodes):
        row, col = edge_index

        value = torch.arange(row.size(0), device=row.device)
        adj_t = SparseTensor(row=col, col=row, value=value,
                             sparse_sizes=(num_nodes, num_nodes))
        
        #Compute the node indices for two-hop angles
        adj_t_row = adj_t[row]
        num_triplets = adj_t_row.set_value(None).sum(dim=1).to(torch.long)

        idx_i = col.repeat_interleave(num_triplets)
        idx_j = row.repeat_interleave(num_triplets)
        idx_k = adj_t_row.storage.col()
        mask = idx_i != idx_k
        idx_i_1, idx_j, idx_k = idx_i[mask], idx_j[mask], idx_k[mask]

        idx_kj = adj_t_row.storage.value()[mask]
        idx_ji_1 = adj_t_row.storage.row()[mask]

        #Compute the node indices for one-hop angles
        adj_t_col = adj_t[col]

        num_pairs = adj_t_col.set_value(None).sum(dim=1).to(torch.long)
        idx_i_2 = row.repeat_interleave(num_pairs)
        idx_j1 = col.repeat_interleave(num_pairs)
        idx_j2 = adj_t_col.storage.col()

        idx_ji_2 = adj_t_col.storage.row()
        idx_jj = adj_t_col.storage.value()

        return idx_i_1, idx_j, idx_k, idx_kj, idx_ji_1, idx_i_2, idx_j1, idx_j2, idx_jj, idx_ji_2


    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        pos = data.pos
        batch = data.batch
        # Initialize node embeddings
        h = torch.index_select(self.embeddings, 0, x.long())

        # Get the edges and pairwise distances in the local layer
        edge_index_l, _ = remove_self_loops(edge_index)
        j_l, i_l = edge_index_l
        dist_l = (pos[i_l] - pos[j_l]).pow(2).sum(dim=-1).sqrt()
        
        # Get the edges pairwise distances in the global layer
        row, col = radius(pos, pos, self.cutoff, batch, batch, max_num_neighbors=500)
        edge_index_g = torch.stack([row, col], dim=0)
        edge_index_g, _ = remove_self_loops(edge_index_g)
        j_g, i_g = edge_index_g
        dist_g = (pos[i_g] - pos[j_g]).pow(2).sum(dim=-1).sqrt()
        
        # Compute the node indices for defining the angles
        idx_i_1, idx_j, idx_k, idx_kj, idx_ji, idx_i_2, idx_j1, idx_j2, idx_jj, idx_ji_2 = self.indices(edge_index_l, num_nodes=h.size(0))

        # Compute the two-hop angles
        pos_ji_1, pos_kj = pos[idx_j] - pos[idx_i_1], pos[idx_k] - pos[idx_j]
        a = (pos_ji_1 * pos_kj).sum(dim=-1)
        b = torch.cross(pos_ji_1, pos_kj).norm(dim=-1)
        angle_1 = torch.atan2(b, a)

        # Compute the one-hop angles
        pos_ji_2, pos_jj = pos[idx_j1] - pos[idx_i_2], pos[idx_j2] - pos[idx_j1]
        a = (pos_ji_2 * pos_jj).sum(dim=-1)
        b = torch.cross(pos_ji_2, pos_jj).norm(dim=-1)
        angle_2 = torch.atan2(b, a)

        # Get the RBF and SBF embeddings
        rbf_g = self.rbf_g(dist_g)
        rbf_l = self.rbf_l(dist_l)
        sbf_1 = self.sbf(dist_l, angle_1, idx_kj)
        sbf_2 = self.sbf(dist_l, angle_2, idx_jj)
        
        rbf_g = self.rbf_g_mlp(rbf_g)
        rbf_l = self.rbf_l_mlp(rbf_l)
        sbf_1 = self.sbf_1_mlp(sbf_1)
        sbf_2 = self.sbf_2_mlp(sbf_2)
        
        # Perform the message passing schemes
        node_sum = 0

        for layer in range(self.n_layer):
            h = self.global_layers[layer](h, rbf_g, edge_index_g)
            h, t = self.local_layers[layer](h, rbf_l, sbf_1, sbf_2, idx_kj, idx_ji, idx_jj, idx_ji_2, edge_index_l)
            node_sum += t
        
        # Readout
        output = global_add_pool(node_sum, batch)
        return output.view(-1)

ModuleNotFoundError: No module named 'torch_geometric'

In [342]:
from torch_geometric.utils import add_self_loops
from torch_scatter import scatter

from utils import MLP, Res, MessagePassing

class Global_MP(MessagePassing):

    def __init__(self, config):
        super(Global_MP, self).__init__()
        self.dim = config.dim

        self.h_mlp = MLP([self.dim, self.dim])

        self.res1 = Res(self.dim)
        self.res2 = Res(self.dim)
        self.res3 = Res(self.dim)
        self.mlp = MLP([self.dim, self.dim])

        self.x_edge_mlp = MLP([self.dim * 3, self.dim])
        self.linear = nn.Linear(self.dim, self.dim, bias=False)

    def forward(self, h, edge_attr, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=h.size(0))

        res_h = h
        
        # Integrate the Cross Layer Mapping inside the Global Message Passing
        h = self.h_mlp(h)
        
        # Message Passing operation
        h = self.propagate(edge_index, x=h, num_nodes=h.size(0), edge_attr=edge_attr)

        # Update function f_u
        h = self.res1(h)
        h = self.mlp(h) + res_h
        h = self.res2(h)
        h = self.res3(h)
        
        # Message Passing operation
        h = self.propagate(edge_index, x=h, num_nodes=h.size(0), edge_attr=edge_attr)

        return h

    def message(self, x_i, x_j, edge_attr, edge_index, num_nodes):
        num_edge = edge_attr.size()[0]

        x_edge = torch.cat((x_i[:num_edge], x_j[:num_edge], edge_attr), -1)
        x_edge = self.x_edge_mlp(x_edge)

        x_j = torch.cat((self.linear(edge_attr) * x_edge, x_j[num_edge:]), dim=0)

        return x_j

    def update(self, aggr_out):

        return aggr_out


class Local_MP(torch.nn.Module):
    def __init__(self, config):
        super(Local_MP, self).__init__()
        self.dim = config.dim

        self.h_mlp = MLP([self.dim, self.dim])

        self.mlp_kj = MLP([3 * self.dim, self.dim])
        self.mlp_ji_1 = MLP([3 * self.dim, self.dim])
        self.mlp_ji_2 = MLP([self.dim, self.dim])
        self.mlp_jj = MLP([self.dim, self.dim])

        self.mlp_sbf1 = MLP([self.dim, self.dim, self.dim])
        self.mlp_sbf2 = MLP([self.dim, self.dim, self.dim])
        self.lin_rbf1 = nn.Linear(self.dim, self.dim, bias=False)
        self.lin_rbf2 = nn.Linear(self.dim, self.dim, bias=False)

        self.res1 = Res(self.dim)
        self.res2 = Res(self.dim)
        self.res3 = Res(self.dim)

        self.lin_rbf_out = nn.Linear(self.dim, self.dim, bias=False)

        self.h_mlp = MLP([self.dim, self.dim])

        self.y_mlp = MLP([self.dim, self.dim, self.dim, self.dim])
        self.y_W = nn.Linear(self.dim, 1)

    def forward(self, h, rbf, sbf1, sbf2, idx_kj, idx_ji_1, idx_jj, idx_ji_2, edge_index, num_nodes=None):
        res_h = h
        
        # Integrate the Cross Layer Mapping inside the Local Message Passing
        h = self.h_mlp(h)
        
        # Message Passing 1
        j, i = edge_index
        m = torch.cat([h[i], h[j], rbf], dim=-1)

        m_kj = self.mlp_kj(m)
        m_kj = m_kj * self.lin_rbf1(rbf)
        m_kj = m_kj[idx_kj] * self.mlp_sbf1(sbf1)
        m_kj = scatter(m_kj, idx_ji_1, dim=0, dim_size=m.size(0), reduce='add')
        
        m_ji_1 = self.mlp_ji_1(m)

        m = m_ji_1 + m_kj

        # Message Passing 2       (index jj denotes j'i in the main paper)
        m_jj = self.mlp_jj(m)
        m_jj = m_jj * self.lin_rbf2(rbf)
        m_jj = m_jj[idx_jj] * self.mlp_sbf2(sbf2)
        m_jj = scatter(m_jj, idx_ji_2, dim=0, dim_size=m.size(0), reduce='add')
        
        m_ji_2 = self.mlp_ji_2(m)

        m = m_ji_2 + m_jj

        # Aggregation
        m = self.lin_rbf_out(rbf) * m
        h = scatter(m, i, dim=0, dim_size=h.size(0), reduce='add')
        
        # Update function f_u
        h = self.res1(h)
        h = self.h_mlp(h) + res_h
        h = self.res2(h)
        h = self.res3(h)

        # Output Module
        y = self.y_mlp(h)
        y = self.y_W(y)

        return h, y

ModuleNotFoundError: No module named 'torch_geometric'

In [343]:
model = TwoLayersNet(nX, nH1, nH2, nY)                           # экземпляр сети        
 
loss      = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),          # параметры модели
                            lr=0.5, momentum=0.8) 

In [334]:
def fit(model, x_train,x_labels, batch_size=100, train=True):    
    model.train(train)                                 # важно для Dropout, BatchNorm
    sumL, sumA, numB = 0, 0, int( len(x_train)/batch_size )  # ошибка, точность, батчей
       
    for i in range(0, numB*batch_size, batch_size):          
        xb = x_train[i: i+batch_size]                          # текущий батч,
        yb = x_labels[i: i+batch_size]                          # X,Y - torch тензоры
              
        y = model(xb)[:,0]                                    # прямое распространение

        L = loss(y, yb)                                  # вычисляем ошибку
  
        if train:                                        # в режиме обучения
            optimizer.zero_grad()                        # обнуляем градиенты        
            L.backward()                                 # вычисляем градиенты            
            optimizer.step()                             # подправляем параметры
                                     
        sumL += L.item()                                 # суммарная ошибка (item из графа)
        sumA += (y.round() == yb).float().mean()         # точность определения класса
         
    return sumL/numB,  sumA/numB 

In [335]:
print( "before:      loss: %.4f accuracy: %.4f" %  fit(model, x_train,x_labels, train=False) )
 
epochs = 1                                            # число эпох
for epoch in tqdm(range(epochs)):                              # эпоха - проход по всем примерам
    L,A = fit(model, x_train, x_labels)                               # одна эпоха
     
    if epoch % 500 == 0 or epoch == epochs-1:                 
        print(f'epoch: {epoch:5d} loss: {L:.4f} accuracy: {A:.4f}' ) 

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11.36it/s]

before:      loss: 0.7721 accuracy: 0.3035
epoch:     0 loss: 3.6688 accuracy: 0.9496


In [336]:
y_pred = model(x_test)

In [337]:
y_pred = y_pred[:,0]

In [338]:
y_pred

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SelectBackward0>)

In [283]:
test = pd.read_csv('Task/test.csv')
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [324]:
test['Active'] = y_pred.detach().numpy()

In [325]:
test.sample()

,Smiles,Active
25,O=c1c2ccccc2nc2n1CCc1c-2[nH]c2ccccc12,0.0


In [253]:
test['Active'] = test['Active'].apply(lambda x: x*1000-33)

In [254]:
test['Active'] = test['Active'].apply(lambda x: int(abs(round(x, 0))))

In [255]:
test['Active'] = test['Active'].apply(lambda x: 1 if x > 1 else x)

In [296]:
test['Active'].value_counts()

0.0    1614
Name: Active, dtype: int64

In [257]:
test.sample(5)

,Smiles,Active
570,N=C1N[C@@H]2[C@H](CCCCC(=O)O)SC[C@@H]2N1,0
1266,N[C@@H](Cc1cc(-c2ccc(Cl)cc2Cl)cc(CP(=O)(O)O)c1...,1
1336,CC[C@@H]1C(C)=NN=C(c2ccc(OC)c(OC)c2)c2cc(OC)c(...,1
618,CN1CCN(CCCN2c3ccccc3Sc3ccc(S(=O)(=O)N(C)C)cc32...,1
1389,COc1ccccc1N1CCN(CCCCN2C(=O)c3ccccc3C2=O)CC1,1


In [258]:
test.to_csv('test2.csv', index=False)